# Service Industry

## Source

For this example we're extracting the table "TOPS19" as shown below (note - preview cropped for reasons of practicality):

In [1]:
from typing import List
from datachef import acquire, preview
from datachef.selection import XlsSelectable

tables: List[XlsSelectable] = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/xls/service-industry.xls")
preview(tables[10], bounded="A1:Q22")


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q
1,TOPSI9,,,UK Production Turnover,,,,,,,,,,,,,
2,,,,Turnover in Production and Services Industries,,,,,,,,,,,,,
3,,,,"Current price, not seasonally adjusted",,,,,,,,,,,,,£ million
4,Back to Contents,,,,,,,Manufacture of air and spacecraft and related machinery,,,,,,,,,
5,,,,,Building of ships and boats,,,,,,Manufacture of other transport equipment,,,,,,
6,,,,,,,,,,,,,,Manufacture of furniture,,,Other manufacturing
7,,,,,,,,,,,,,,,,,
8,,,,,30.1,,,30.3,,,30.2/4/9 (30OTHER),,,31,,,32
9,,,,,JQR4,,,JQS8,,,JQU4,,,JQV2,,,JQV5
10,,,,,,,,,,,,,,,,,


From an xlsx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/xls/servie-industry.xls).

# Requirements

- We'll take the line 4 and 5 headers as "Product".
- We'll call "Year" from column A and clean it up.
- We'll take "Quarter" from column B.
- We'll take row 9 as "CDID" (as I happen to know that's the name of this particular type of identifier).
- We'll call the observations column "Value"

this is a pretty standard recipe save perhaps the badly merged cells (lines 4, 5 & 6).

In [2]:
from typing import List
from datachef import acquire, against, filters, preview
from datachef.direction import up, down, left, right
from datachef.output import Column, TidyData
from datachef.selection import XlsSelectable

tables: List[XlsSelectable] = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/xls/service-industry.xls")

# Do sensible things
table = tables[10]
wanted_table = "TOPSI9"
assert table.name == wanted_table, f'Got table {table.name}, expected {wanted_table}'

anchor = table.re(".*ships and boats.*").assert_one().shift(left).label_as("Anchor Cell")
year = anchor.shift(left(3)).expand(down).is_not_blank().label_as("Year")
quarter = (anchor.shift(left(2)).expand(down).is_not_blank() | year.shift(right)).label_as("Quarter")
cdid = table.re("^[A-Z]{3}\d$").assert_single_row().label_as("CDID")
product = anchor.extrude(up).extrude(down).expand(right).is_not_blank().label_as("Product")
observations = cdid.waffle(down, year | quarter).label_as("Value")

preview(anchor, observations, product, year, quarter, cdid, bounded="A1:Q22")

tidy_data = TidyData(
    observations,
    Column(product.finds_observations_directly(down)),
    Column(year.finds_observations_closest(down), apply=lambda x: x[:4], validate=against.is_numeric),
    Column(quarter.finds_observations_directly(right), apply=lambda x: "All" if x == "" else x),
    Column(cdid.finds_observations_directly(down))
)

print(tidy_data)

ReferenceOutsideSelectionError: 
                You cannot make a selection of "A" at
                this time. One or more cells of these cells
                does not exist in your CURRENT SELECTION.
                
                If you believe they should be, you need to check
                your sequencing.

                As an example: if you filter a selection to just
                column A then try excel_ref('B') you'll get
                this error.

                Where practical, you can debug the selected cells
                at any given time with <selection>.print_excel_refs()
                